### Notebook to create BLOCK-T186 for LUT in Azimuth with closed loop

Created on: 2024-10-09

Author: Guillem Megias

In [ ]:
from lsst.ts.observing import ObservingBlock, ObservingScript 
from lsst.ts.aos.analysis import build_configuration_schema
import os

In [ ]:
current_path = os.getcwd()
block_number = 'T186'
program = "BLOCK-T186"
reason = "SITCOM-1490"
note = "LUT_azimuth_closed_loop"
constraints = []

### Define configuration schema

In [ ]:
# Define the configurable properties that we will use in the configuration schema
properties = {
    "run_filter": {
        "description": "ComCam filter to use.",
        "type": "string",
        "default": "r_03"
    },
    "run_elevation": {
        "description": "Elevation to use for the elevation sweep.",
        "type": "number",
        "default": 80
    },
    "run_dofs": {
        "description": "Degrees of freedom to use for the elevation sweep.",
        "type": "array",
        "items": {
            "type": "integer",
            "minimum": 0,
            "maximum": 49,
        },
        "default": list(range(0, 50))
    },
    "run_maxiter": {
        "description": "Maximum number of iterations for the elevation sweep.",
        "type": "integer",
        "default": 5
    }
}

# Build the configuration schema for BLOCK-404
configuration_schema = build_configuration_schema(block_number, properties)
print(configuration_schema)

### Define scripts and block

In [ ]:
comcam_closedloop_script = ObservingScript(
    name="maintel/close_loop_comcam.py",
    standard=True,
    parameters= dict(
        filter="$run_filter",
        program="$program",
        reason=reason,
        note=note,
        exposure_time=30.0,
        max_iter="$run_maxiter",
        used_dofs="$run_dofs",
    )
)

stop_tracking_script = ObservingScript(
    name="maintel/stop_tracking.py",
    standard=True,
    parameters = dict()
)

In [ ]:
azimuth_positions = [-180, -150, -120, -90, -60, -30, 0, 30, 60, 90, 120, 150, 180]
scripts = []

for azimuth in azimuth_positions:
    track_target_script = ObservingScript(
        name="maintel/track_target.py",
        standard=True,
        parameters = dict(
            track_azel = dict(
                az = azimuth,
                el = '$run_elevation'
            )
        )
    )

    scripts.append(track_target_script)
    scripts.append(comcam_closedloop_script)
scripts.append(stop_tracking_script)

In [ ]:
block = ObservingBlock(
    name = program,
    program = program,
    configuration_schema=configuration_schema,
    scripts = scripts,
)

### Save configurable block

In [ ]:
block.model_dump_json(indent=2)

output_file_path = f'{current_path}/aos/ts_config_ocs/Scheduler/observing_blocks_maintel/AOS/LUTs/{program}.json'

with open(output_file_path, 'w') as file:
    file.write(block.model_dump_json(indent=2))